# <center>Instituto Politécnico Nacional</center>
### <center>Escuela Superior de Cómputo</center>
## <center>Proyecto Reglas de Asociación - Python</center>
#### <center>Unidad de aprendizaje: Data Mining</center>
#### <center>Grupo: 3CV6</center>
##### <center>Alumno(a): Ramos Diaz Enrique</center>
##### <center>Profesor(a): Ocampo Botello Fabiola</center>
<br><br><br><br>

# 1.- Enunciado del problema

Este es un conjunto de datos transnacionales que contiene todas las transacciones que ocurren entre el 01/12/2010 y el 09/12/2011 para un comercio minorista en línea registrado en el Reino Unido y sin tienda. La compañía vende principalmente regalos únicos para toda ocasión. Muchos clientes de la empresa son mayoristas.

**El objetivo es encontrar qué artículos compran los clientes franceses a partir de otros previamente adquiridos**, con ayuda de los registros históricos del comercio.

# 2.- Diccionario de datos

![groceries-1](dicc1.jpg)
![groceries-2](dicc2.jpg)

# 3.- Importar las librerias

Primero es necesario importar algunas librerías que nos proporcionaran métodos o funciones para construir modelos utilizando reglas de asociación. Las principales son ***pandas*** y ***numpy***, que se utilizan para todo el preprocesamiento de los datos; y  ***mlxtend***, que integra las herramientas para aplicar técnicas de reglas de asociación y encontrar itemsets frequentes en un conjunto de datos.

In [1]:
!pip install mlxtend
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 4.- Desarrollo: Proceso KDD

## 4.1.- Carga de la base de datos

La base de datos a utilizar tiene como nombre *groceries.csv*. 
Utilizamos el método de pandas llamado **read_csv()** y mostramos la información y descripción de los atributos con **info()** y **describe()**.

In [2]:
data=pd.read_csv("groceries.csv")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.0,United Kingdom


### 4.1.1.- Información y descripción de los datos

In [3]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


## 4.2.- Limpieza de datos

Si revisamos la existencia de datos nulos con el método **isnull().sum()**, notaremos que existen datos nulos en los atributos *Description* y *CustomerID*. Para este proyecto, los eliminaremos.

In [4]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
data = data.dropna()

Según el diccionario de datos, en los registros del atributo *InvoiceNo* existen transacciones que fueron canceladas y otros más que no poseen número de factura, por los que los eliminamos igualmente.

In [6]:
# Quitar transacciones sin número de factura
data['InvoiceNo'] = data['InvoiceNo'].astype('str') 
  
# Quitar todas las transacciones canceladas
data = data[~data['InvoiceNo'].str.contains('C')] 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      397924 non-null object
StockCode      397924 non-null object
Description    397924 non-null object
Quantity       397924 non-null int64
InvoiceDate    397924 non-null object
UnitPrice      397924 non-null float64
CustomerID     397924 non-null float64
Country        397924 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 27.3+ MB


Por último, eliminamos los registros de los campos *UnitPrice* y *Quantity* cuyo valor sea negativo.

In [7]:
data = data[data["UnitPrice"] >= 0]
data = data[data["Quantity"] >= 0]
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      397924 non-null object
StockCode      397924 non-null object
Description    397924 non-null object
Quantity       397924 non-null int64
InvoiceDate    397924 non-null object
UnitPrice      397924 non-null float64
CustomerID     397924 non-null float64
Country        397924 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 27.3+ MB


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.0,United Kingdom


## 4.3.- Integración y selección de los datos

Para el objetivo de este proyecto, solamente se van a tomar las transacciones cuyos clientes residen en Francia, por lo que aplicamos ese filtro por medio del atributo *Country*.

In [8]:
basket = data[data["Country"] == "France"]
basket = basket.drop("Country", axis=1)
basket.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 26 to 541908
Data columns (total 7 columns):
InvoiceNo      8342 non-null object
StockCode      8342 non-null object
Description    8342 non-null object
Quantity       8342 non-null int64
InvoiceDate    8342 non-null object
UnitPrice      8342 non-null float64
CustomerID     8342 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 521.4+ KB


## 4.4.- Transformación de los datos

Eliminamos los espacios en blanco extras de los registros del atributo *Description*.

In [9]:
# Quitando espacios en blanco extras
basket['Description'] = basket['Description'].str.strip() 
basket.info()
basket.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 26 to 541908
Data columns (total 7 columns):
InvoiceNo      8342 non-null object
StockCode      8342 non-null object
Description    8342 non-null object
Quantity       8342 non-null int64
InvoiceDate    8342 non-null object
UnitPrice      8342 non-null float64
CustomerID     8342 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 521.4+ KB


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24,01/12/2010 08:45,3.75,12583.0
27,536370,22727,ALARM CLOCK BAKELIKE RED,24,01/12/2010 08:45,3.75,12583.0
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,01/12/2010 08:45,3.75,12583.0
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,01/12/2010 08:45,0.85,12583.0
30,536370,21883,STARS GIFT TAPE,24,01/12/2010 08:45,0.65,12583.0


Es necesario cambiar el formato del conjunto de datos para poder aplicar los algoritmos de reglas de asociación.
En primer lugar, hay que agrupar los datos con base al ***número de factura***, y en cada factura se agrupan los ***productos o artículos*** existentes. 

Luego, rellenamos el nuevo DataFrame con la ***cantidad de productos*** por cada transaccion, y nos olvidamos del resto de los atributos.

De esta forma, podemos tener una mejor ***descripción de los productos*** y la ***cantidad*** de ellos en cada transacción por separado (los productos son los que nos interesan para generar los itemsets y reglas de asociación).


In [10]:
basket = (basket.groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
basket.info()
basket.head()

<class 'pandas.core.frame.DataFrame'>
Index: 389 entries, 536370 to 581587
Columns: 1543 entries, 10 COLOUR SPACEBOY PEN to ZINC T-LIGHT HOLDER STARS SMALL
dtypes: float64(1543)
memory usage: 4.6+ MB


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Adicionalmente a esto, programamos un pequeño método que identifica qué productos estan presentes en cada transacción, según la ***cantidad*** de estos presentes en ellas, codificandolos como parte de estas o no y así tener una **respresentación binaria de la base de datos**: 

0 = El producto no forma parte de la transacción.

1 = El producto forma parte de la transacción.

In [11]:
def bin_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
  
# Codificando los productos de cada transacción
basket = basket.applymap(bin_encode) 
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 4.5.- Minería de datos

### 4.5.1.- Extracción de itemsets frecuentes (2 - Itemset)

In [12]:
# Building the model 
frq_items2 = apriori(basket, min_support = 0.02, use_colnames = True, max_len=2)
frq_items2

,support,itemsets
0,0.030848,(10 COLOUR SPACEBOY PEN)
1,0.023136,(12 PENCILS TALL TUBE RED RETROSPOT)
2,0.035990,(3 PIECE SPACEBOY COOKIE CUTTER SET)
3,0.046272,(36 PENCILS TUBE RED RETROSPOT)
4,0.023136,(36 PENCILS TUBE WOODLAND)
...,...,...
1206,0.028278,"(SPACEBOY LUNCH BOX, STRAWBERRY LUNCH BOX WITH..."
1207,0.030848,"(WOODLAND MINI BACKPACK, SPACEBOY MINI BACKPACK)"
1208,0.020566,"(TEA PARTY BIRTHDAY CARD, STRAWBERRY LUNCH BOX..."
1209,0.020566,"(WATERING CAN PINK BUNNY, STRAWBERRY LUNCH BOX..."


### 4.5.2.- Aplicación de reglas de asociacion (2 - Itemset)

In [13]:
# Collecting the inferred rules in a dataframe 
itemset2 = association_rules(frq_items2, metric = "lift", min_threshold = 0) 
itemset2 = itemset2.sort_values(['lift', 'support', 'confidence'], ascending =[True, False, False]) 
itemset2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
205,(ASSORTED COLOUR BIRD ORNAMENT),(POSTAGE),0.043702,0.771208,0.020566,0.470588,0.610196,-0.013138,0.432162
204,(POSTAGE),(ASSORTED COLOUR BIRD ORNAMENT),0.771208,0.043702,0.020566,0.026667,0.610196,-0.013138,0.982498
8,(POSTAGE),(36 PENCILS TUBE RED RETROSPOT),0.771208,0.046272,0.025707,0.033333,0.720370,-0.009979,0.986615
9,(36 PENCILS TUBE RED RETROSPOT),(POSTAGE),0.046272,0.771208,0.025707,0.555556,0.720370,-0.009979,0.514781
479,(CLOTHES PEGS RETROSPOT PACK 24),(POSTAGE),0.041131,0.771208,0.023136,0.562500,0.729375,-0.008584,0.522953
...,...,...,...,...,...,...,...,...,...
1644,(RECYCLED ACAPULCO MAT TURQUOISE),(RECYCLED ACAPULCO MAT PINK),0.030848,0.025707,0.020566,0.666667,25.933333,0.019773,2.922879
1640,(RECYCLED ACAPULCO MAT LAVENDER),(RECYCLED ACAPULCO MAT RED),0.025707,0.030848,0.023136,0.900000,29.175000,0.022343,9.691517
1642,(RECYCLED ACAPULCO MAT LAVENDER),(RECYCLED ACAPULCO MAT TURQUOISE),0.025707,0.030848,0.023136,0.900000,29.175000,0.022343,9.691517
1641,(RECYCLED ACAPULCO MAT RED),(RECYCLED ACAPULCO MAT LAVENDER),0.030848,0.025707,0.023136,0.750000,29.175000,0.022343,3.897172


### 4.5.3.- Extracción de itemsets frecuentes (3 - Itemset)

In [14]:
# Building the model 
frq_items3 = apriori(basket, min_support = 0.02, use_colnames = True, max_len=3)
frq_items3

,support,itemsets
0,0.030848,(10 COLOUR SPACEBOY PEN)
1,0.023136,(12 PENCILS TALL TUBE RED RETROSPOT)
2,0.035990,(3 PIECE SPACEBOY COOKIE CUTTER SET)
3,0.046272,(36 PENCILS TUBE RED RETROSPOT)
4,0.023136,(36 PENCILS TUBE WOODLAND)
...,...,...
2105,0.025707,"(SET/6 RED SPOTTY PAPER PLATES, SET/10 RED POL..."
2106,0.100257,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET..."
2107,0.020566,"(SET/20 RED RETROSPOT PAPER NAPKINS, SPACEBOY ..."
2108,0.020566,"(SET/6 RED SPOTTY PAPER PLATES, SPACEBOY BIRTH..."


### 4.5.4.- Aplicación de reglas de asociacion (3 - Itemset)

In [15]:
# Collecting the inferred rules in a dataframe 
itemset3 = association_rules(frq_items3, metric = "lift", min_threshold = 0) 
itemset3 = itemset3.sort_values(['lift', 'support', 'confidence'], ascending =[True, False, False]) 
itemset3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
205,(ASSORTED COLOUR BIRD ORNAMENT),(POSTAGE),0.043702,0.771208,0.020566,0.470588,0.610196,-0.013138,0.432162
204,(POSTAGE),(ASSORTED COLOUR BIRD ORNAMENT),0.771208,0.043702,0.020566,0.026667,0.610196,-0.013138,0.982498
4476,"(LUNCH BAG RED RETROSPOT, LUNCH BAG DOLLY GIRL...",(POSTAGE),0.043702,0.771208,0.023136,0.529412,0.686471,-0.010567,0.486183
4477,(POSTAGE),"(LUNCH BAG RED RETROSPOT, LUNCH BAG DOLLY GIRL...",0.771208,0.043702,0.023136,0.030000,0.686471,-0.010567,0.985874
3567,(POSTAGE),"(DOLLY GIRL LUNCH BOX, LUNCH BAG RED RETROSPOT)",0.771208,0.038560,0.020566,0.026667,0.691556,-0.009173,0.987780
...,...,...,...,...,...,...,...,...,...
1643,(RECYCLED ACAPULCO MAT TURQUOISE),(RECYCLED ACAPULCO MAT LAVENDER),0.030848,0.025707,0.023136,0.750000,29.175000,0.022343,3.897172
6647,"(POSTAGE, RECYCLED ACAPULCO MAT RED)",(RECYCLED ACAPULCO MAT LAVENDER),0.023136,0.025707,0.020566,0.888889,34.577778,0.019971,8.768638
7128,"(RECYCLED ACAPULCO MAT RED, RECYCLED ACAPULCO ...",(RECYCLED ACAPULCO MAT LAVENDER),0.023136,0.025707,0.020566,0.888889,34.577778,0.019971,8.768638
6650,(RECYCLED ACAPULCO MAT LAVENDER),"(POSTAGE, RECYCLED ACAPULCO MAT RED)",0.025707,0.023136,0.020566,0.800000,34.577778,0.019971,4.884319


# 5.- Resultados encontrados